In [18]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Data prep

1. Get data
2. Split it into sequences of length $k$.
3. One-hot encode sequences based on your vocabulary, e.g. $10$.
4. Batch the data so the dataset has shape `(batch_size, seq_len, encoding_len)`.
5. Send them to the device.

In [15]:
vocab_size = 10

x_train, y_train = [], []
x_test, y_test = [], []

# Model definition

In [19]:
class LitRnn(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, n_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        hidden_state = self.init_hidden_state(batch_size)
        out, hidden_state = self.rnn(x, hidden_state)
        print(out.shape)
        
        # Why is this reshaping necessary?
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.linear(out)
        
        return out, hidden_state
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        out, hidden = self.forward(x)
        loss = F.CrossEntropyLoss(out, y)
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimiser = torch.optim.SGD(self.parameters(), lr=1e-3)
        return optimiser

In [20]:
simple_rnn = LitRnn(input_size=vocab_size, 
                    hidden_size=12, 
                    output_size=vocab_size,
                    n_layers=3)

trainer = pl.Trainer()
trainer.fit(model=simple_rnn, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


NameError: name 'train_loader' is not defined

# Training procedure

In [17]:
# Hyperparameters.
num_epochs = 100
lr = 0.01

# Model.
simple_rnn = SimpleRnn(input_size=vocab_size, hidden_size=12, output_size=vocab_size, n_layers=2)

# Loss and optimiser.
loss = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(simple_rnn.parameters(), lr=lr)

for epoch in range(num_epochs):
    optimiser.zero_grad()
    output = simple_rnn(x_train)
    l = loss(output, y_train)
    l.backward()
    optimiser.step()
    log(epoch, l)

AttributeError: 'list' object has no attribute 'size'